# 🌩️ **CloudLeecher: Production Backend**

Welcome to the **CloudLeecher** backend. This notebook turns your Google Colab instance into a powerful, high-speed torrent downloader that saves files directly to your Google Drive.

### **Instructions**
1.  **Mount Drive**: Connect your Google storage.
2.  **Install**: Set up the environment.
3.  **Start Services**: Launch the backend and get your public connection URL.
4.  **Connect**: Paste the URL into the CloudLeecher Frontend.

## 1. 📂 **Mount Google Drive**
We need access to your Drive to save the downloaded files.

In [ ]:
from google.colab import drive
import os

# Mount Google Drive
drive.mount('/content/drive')

# Define and Create Download Directory
DOWNLOAD_DIR = "/content/drive/MyDrive/TorrentDownloads"
os.makedirs(DOWNLOAD_DIR, exist_ok=True)

print(f"✅ Download Directory Ready: {DOWNLOAD_DIR}")

## 2. 🛠️ **Install Dependencies**
Installing `aria2` (the download engine), `flask` (the API server), and `pyngrok` (for public access).

In [ ]:
%%capture
!apt-get update -qq
!apt-get install -y -qq aria2
!pip install -q flask flask-cors pyngrok

print("✅ All dependencies installed successfully.")

## 3. 🚀 **Start Downloader Service**
Initializing the Aria2 RPC server in the background.

In [ ]:
import subprocess

# Start Aria2c as a daemon process
cmd = [
    "aria2c",
    "--enable-rpc",
    "--rpc-listen-all=true",
    "--rpc-allow-origin-all",
    f"--dir={DOWNLOAD_DIR}",
    "--file-allocation=none",
    "--max-connection-per-server=4",
    "--seed-time=0",
    "--daemon=true"
]

subprocess.run(
    cmd,
    stdout=subprocess.DEVNULL,
    stderr=subprocess.DEVNULL
)

print("✅ Aria2 Background Service Started.")

## 4. 📝 **Create API Backend**
Generating the `app.py` file which serves as the brain of CloudLeecher.

In [ ]:
%%writefile app.py
import xmlrpc.client
from flask import Flask, request, jsonify
from flask_cors import CORS
import os
import shutil
import base64

app = Flask(__name__)
CORS(app)

# Configuration
DOWNLOAD_DIR = "/content/drive/MyDrive/TorrentDownloads"
ARIA2_RPC_URL = "http://localhost:6800/rpc"

# Connect to Aria2 RPC
s = xmlrpc.client.ServerProxy(ARIA2_RPC_URL)

@app.route('/health', methods=['GET'])
def health():
    return jsonify({"status": "ok", "service": "CloudLeecher-Backend"})

@app.route('/api/download/magnet', methods=['POST'])
def add_magnet():
    data = request.json
    magnet_link = data.get('magnet')
    if not magnet_link:
        return jsonify({"error": "Magnet link is required"}), 400
    try:
        gid = s.aria2.addUri([magnet_link])
        return jsonify({"status": "success", "gid": gid})
    except Exception as e:
        return jsonify({"error": str(e)}), 500

@app.route('/api/download/file', methods=['POST'])
def add_torrent_file():
    try:
        data = request.json
        # Content should be base64 encoded string
        b64_content = data.get('torrent')
        if not b64_content:
            return jsonify({"error": "Torrent file content is required"}), 400

        # Decode base64 to bytes, then wrap in xmlrpc.client.Binary
        # Aria2 via RPC expects a base64 encoded string as text, BUT via standard XML-RPC
        # typing, passing a string sends <string>. Passing Binary sends <base64>.
        # Aria2's documentation implies it wants the Base64 value... but practice shows
        # it often requires the Binary wrapper so the XML-RPC server treats it correctly.
        # Let's try wrapping it as a Binary object.

        raw_bytes = base64.b64decode(b64_content)
        binary_torrent = xmlrpc.client.Binary(raw_bytes)

        gid = s.aria2.addTorrent(binary_torrent)
        return jsonify({"status": "success", "gid": gid})
    except Exception as e:
        return jsonify({"error": str(e)}), 500

@app.route('/api/status', methods=['GET'])
def get_status():
    try:
        # Fetch status for different categories of downloads
        keys = ["gid", "status", "totalLength", "completedLength", "downloadSpeed", "dir", "files"]
        active = s.aria2.tellActive(keys)
        waiting = s.aria2.tellWaiting(0, 100, keys)
        stopped = s.aria2.tellStopped(0, 100, keys)
        return jsonify({
            "active": active,
            "waiting": waiting,
            "stopped": stopped
        })
    except Exception as e:
        return jsonify({"error": str(e)}), 500

@app.route('/api/control/pause', methods=['POST'])
def pause_download():
    try:
        gid = request.json.get('gid')
        s.aria2.pause(gid)
        return jsonify({"status": "paused", "gid": gid})
    except Exception as e:
        return jsonify({"error": str(e)}), 500

@app.route('/api/control/resume', methods=['POST'])
def resume_download():
    try:
        gid = request.json.get('gid')
        s.aria2.unpause(gid)
        return jsonify({"status": "resumed", "gid": gid})
    except Exception as e:
        return jsonify({"error": str(e)}), 500

@app.route('/api/control/remove', methods=['POST'])
def remove_download():
    try:
        gid = request.json.get('gid')
        s.aria2.forceRemove(gid)
        return jsonify({"status": "removed", "gid": gid})
    except Exception as e:
        return jsonify({"error": str(e)}), 500

@app.route('/api/drive/info', methods=['GET'])
def drive_info():
    try:
        total, used, free = shutil.disk_usage(DOWNLOAD_DIR)
        return jsonify({
            "total": total,
            "used": used,
            "free": free
        })
    except Exception as e:
        # Fallback if path doesn't exist yet
        return jsonify({"total": 0, "used": 0, "free": 0})

if __name__ == "__main__":
    app.run(port=5000)

## 5. 🌐 **Launch Public Server**
Starting the application and generating your public access URL.

> **⚠️ Important**: Ensure you have added your Ngrok Authtoken to Colab Secrets with the key `NGROK-AUTHTOKEN`.

In [ ]:
from pyngrok import ngrok
from google.colab import userdata
import subprocess
import sys
import time
import os

# 1. Authenticate Ngrok
try:
    AUTH_TOKEN = userdata.get("NGROK-AUTHTOKEN")
    ngrok.set_auth_token(AUTH_TOKEN)
except Exception as e:
    print("❌ Error: Ngrok Auth Token not found! Please add 'NGROK-AUTHTOKEN' to Colab Secrets (Key icon on the left).")
    raise e

# 2. Cleanup Old Processes (Port 5000)
ngrok.kill()
os.system("fuser -k 5000/tcp > /dev/null 2>&1")

# 3. Start Flask App in Background
log_file = open("flask.log", "w")
subprocess.Popen([sys.executable, "app.py"], stdout=log_file, stderr=log_file)
time.sleep(3)  # Allow Flask to initialize

# 4. Open Ngrok Tunnel
try:
    public_url = ngrok.connect(5000).public_url
    print("\n" + "="*60)
    print(f"🔗 PUBLIC URL: {public_url}")
    print("="*60 + "\n")
    print("✅ CloudLeecher Backend is Online!")
    print("🌍 Frontend App: https://cloudleecher.web.app")
    print("📋 Copy the URL above (PUBLIC URL) and paste it into the CloudLeecher Frontend app.")

    # Keep cell running to keep thread alive
    while True:
        time.sleep(10)
except Exception as e:
    print(f"❌ Failed to start Ngrok: {e}")